# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
#print(file_path_list)
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821




## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.



### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
##Query 1 requirements:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#Query Description:Create table table for the first query. We use sessionId and itemInSession in the primary because both are required for our query.
query = "CREATE TABLE IF NOT EXISTS music_history "
query = query + "(sessionId int,itemInSession int,artist text,firstName text,\
                 gender text,lastName text,length float,level text,\
                 location text,song text,userId int,\
                 PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [11]:
## Insert all records from event_data_file_new.csv into the table

file = 'event_datafile_new.csv'
query = "INSERT INTO music_history (sessionId,itemInSession,artist,firstName,\
         gender,lastName,length,level,location,song,userId)"
query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[1], 
                                line[2], line[4], float(line[5]), line[6], 
                                line[7], line[9], int(line[10])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Query Description:Execute query 1 and check that the expeted reult is returned back.
query = "SELECT artist,song,length FROM music_history WHERE sessionId=338 and itemInSession  = 4"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [13]:
## Query 2 requirements: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Query Description:I use a composite partition key compose of userid and sessionId because both are required for our query. 
## I added the itemInSession as clustering key because we have to sort the results using the itemInSession. 
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int,sessionId int,itemInSession int,artist text,\
                 firstName text,gender text,lastName text,\
                 length float,level text,location text,song text,\
                 PRIMARY KEY ((userid,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## After creating the table song_playlist_session,Insert all records from the csv file 
    file = 'event_datafile_new.csv'
query = "INSERT INTO song_playlist_session (userId,sessionId,itemInSession,\
         artist,firstName,gender,lastName,length,level,location,song)"
query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]),
                                line[0], line[1], line[2],line[4], 
                                float(line[5]), line[6], line[7],
                                line[9]))
       
        

## Query Description: Execute query 2 and check that the expeted result is returned back.
query = "SELECT artist,song,firstName,lastName FROM song_playlist_session \
WHERE userid = 10 and sessionid = 182"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)        
                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [14]:
## Query 3 requirements: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## I use song and userid as the primary to uniquely identify which songs have been listened by each user.

query = "CREATE TABLE IF NOT EXISTS music_history_user \
         (song text,userId int,artist text,firstName text,gender text,\
         itemInSession int,lastName text,length float,level text,\
         location text,sessionId int,PRIMARY KEY (song,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## After creating the table song_playlist_session,Insert all records from the csv file 
    file = 'event_datafile_new.csv'
query = "INSERT INTO music_history_user (song,userId,artist,firstName,gender,\
         itemInSession,lastName,length,level,location,sessionId)"
query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:        
        session.execute(query, (line[9],int(line[10]),line[0], line[1],
                                line[2], int(line[3]),line[4], 
                                float(line[5]), line[6], line[7], 
                                int(line[8])))
        
        
## Query Description: Execute query 3 and check that the expeted result is returned back.
query = "SELECT firstName,lastName FROM music_history_user \
         WHERE song='All Hands Against His Own'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)                          

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [15]:
## Drop the table before closing out the sessions

In [16]:
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)  
    
query = "drop table music_history_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()